### This file is aimed at containing the method for loading our models

In [ ]:
# The 2nd slash shows the performed quantization

model_names = {
    "legendhasit/xgen-7b-8k-inst-8bit/4bit",
    "legendhasit/xgen-7b-8k-inst-8bit",
    "Salesforce/xgen-7b-8k-inst",
    "mosaicml/mpt-7b-instruct", # A quantizer
    "Trelis/mpt-7b-instruct-hosted-inference-8bit", # Unreliable
    "mosaicml/mpt-7b-8k-instruct", # A quantizer
    "mosaicml/mpt-30b-instruct", # A quantizer
    

}

# Pas retenus: BARThez et compagnie

#### XGen

In [ ]:
# Prompt templates
prompt_template_XGen = "A chat between a curious human and an artificial intelligence assistant.\nThe assistant gives helpful, detailed, and polite answers to the human's questions.\n\n### Human: {instruction}\n\n### Assistant: "

prompt_templates["legendhasit/xgen-7b-8k-inst-8bit"] = prompt_template_XGen
prompt_templates["legendhasit/xgen-7b-8k-inst-8bit/4bit"] = prompt_template_XGen
prompt_templates["Salesforce/xgen-7b-8k-inst"] = prompt_template_XGen

# Context lengths # Allow to know whether the model generated out of his context window
context_length_XGen = 8000 

context_lengths["legendhasit/xgen-7b-8k-inst-8bit"] = context_length_XGen
context_lengths["legendhasit/xgen-7b-8k-inst-8bit/4bit"] = context_length_XGen
context_lengths["Salesforce/xgen-7b-8k-inst"] = context_length_XGen

# Tokenizers
def get_tokenizer_XGen8bit(): # Le même pour le 4bit
    return AutoTokenizer.from_pretrained('legendhasit/xgen-7b-8k-inst-8bit', trust_remote_code=True)

def get_tokenizer_XGen():
    return AutoTokenizer.from_pretrained('Salesforce/xgen-7b-8k-inst', trust_remote_code=True)

tokenizers["legendhasit/xgen-7b-8k-inst-8bit"] = get_tokenizer_XGen8bit()
tokenizers["legendhasit/xgen-7b-8k-inst-8bit/4bit"] = get_tokenizer_XGen8bit()
tokenizers["Salesforce/xgen-7b-8k-inst"] = get_tokenizer_XGen()

# Models

def get_model_XGen8bit():
    model = AutoModelForCausalLM.from_pretrained(
    'legendhasit/xgen-7b-8k-inst-8bit',
    device_map="auto",
    trust_remote_code=True
    )
    return model

def get_model_XGen4bit():
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    )
    model = AutoModelForCausalLM.from_pretrained(
    'legendhasit/xgen-7b-8k-inst-8bit',
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
    )
    return model

def get_model_XGen():
    model = AutoModelForCausalLM.from_pretrained(
    'Salesforce/xgen-7b-8k-inst',
    device_map="auto",
    trust_remote_code=True
    )
    return model

models["legendhasit/xgen-7b-8k-inst-8bit"] = get_model_XGen8bit
models["legendhasit/xgen-7b-8k-inst-8bit/4bit"] = get_model_XGen4bit
models["Salesforce/xgen-7b-8k-inst"] = get_model_XGen

# Tokenize (if specificities in the way models call the tokenizer)

# Inference (if specificities in the way models call the generate function)

# Treating the output (to remove the input if present in the output as well as the end of text token for example)

def treat_output_XGen(output): # Differs if it is N-shot. Here, it is 0-shot
    occ_1 = output.find("### Assistant: ")
    output = output[occ_1+15:]
    if output.find('<|endoftext|>')!=-1:
        output = output[:-14]
    return output

treat_output['legendhasit/xgen-7b-8k-inst-8bit'] = treat_output_XGen
treat_output['legendhasit/xgen-7b-8k-inst-8bit/4bit'] = treat_output_XGen
treat_output['Salesforce/xgen-7b-8k-inst'] = treat_output_XGen


#### MPT

In [ ]:
"mosaicml/mpt-7b-instruct", # A quantizer
"Trelis/mpt-7b-instruct-hosted-inference-8bit", # Unreliable
"mosaicml/mpt-7b-8k-instruct", # A quantizer
"mosaicml/mpt-30b-instruct", # A quantizer

"mosaicml/mpt-7b-instruct"
"Trelis/mpt-7b-instruct-hosted-inference-8bit"
"mosaicml/mpt-7b-8k-instruct"
"mosaicml/mpt-30b-instruct"

In [ ]:
# Prompt templates
prompt_template_MPT7B = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:
"""

prompt_template_MPT30B = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n" # On considère que c'est le même template que pour MPT 7B 8k car pour ce dernier, le template utilisé n'est pas précisé

prompt_templates["mosaicml/mpt-7b-instruct"] = prompt_template_MPT7B
prompt_templates["Trelis/mpt-7b-instruct-hosted-inference-8bit"] = prompt_template_MPT7B
prompt_templates["mosaicml/mpt-7b-8k-instruct"] = prompt_template_MPT30B
prompt_templates["mosaicml/mpt-30b-instruct"] = prompt_template_MPT30B

# Context lengths # Allow to know whether the model generated out of his context window
context_length_MPT7B8k = 8192
context_length_MPT30B = 8192 # Even 16k theoretically, but 2048 is safer
context_length_MPT7B = 2048

context_lengths["mosaicml/mpt-7b-instruct"] = context_length_MPT7B
context_lengths["Trelis/mpt-7b-instruct-hosted-inference-8bit"] = context_length_MPT7B
context_lengths["mosaicml/mpt-7b-8k-instruct"] = context_length_MPT7B8k
context_lengths["mosaicml/mpt-30b-instruct"] = context_length_MPT30B

# Tokenizers
def get_tokenizer_MPT7B(): # Le même pour le 4bit
    return AutoTokenizer.from_pretrained('mosaicml/mpt-7b-instruct', trust_remote_code=True)

def get_tokenizer_MPT30B():
    return AutoTokenizer.from_pretrained('mosaicml/mpt-30b-instruct', trust_remote_code=True)

def get_tokenizer_MPT7B8k():
    return AutoTokenizer.from_pretrained('mosaicml/mpt-7b-8k-instruct', trust_remote_code=True)

tokenizers["mosaicml/mpt-7b-instruct"] = get_tokenizer_MPT7B
tokenizers["Trelis/mpt-7b-instruct-hosted-inference-8bit"] = get_tokenizer_MPT7B
tokenizers["mosaicml/mpt-7b-8k-instruct"] = get_tokenizer_MPT7B8k
tokenizers["mosaicml/mpt-30b-instruct"] = get_tokenizer_MPT30B

# Models

def get_model_MPT7B():
    model = AutoModelForCausalLM.from_pretrained(
    'mosaicml/mpt-7b-instruct',
    device_map="auto",
    trust_remote_code=True
    )
    return model

def get_model_MPT7B8bit():
    model = AutoModelForCausalLM.from_pretrained(
    'Trelis/mpt-7b-instruct-hosted-inference-8bit',
    device_map="auto",
    trust_remote_code=True
    )
    return model

def get_model_MPT7B8k():

    model = AutoModelForCausalLM.from_pretrained(
    'mosaicml/mpt-7b-8k-instruct',
    device_map="auto",
    trust_remote_code=True
    )
    return model

def get_model_MPT30B():
    model = AutoModelForCausalLM.from_pretrained(
    'mosaicml/mpt-30b-instruct',
    device_map="auto",
    trust_remote_code=True
    )
    return model

models["mosaicml/mpt-7b-instruct"] = get_model_MPT7B
models["Trelis/mpt-7b-instruct-hosted-inference-8bit"] = get_model_MPT7B8bit
models["mosaicml/mpt-7b-8k-instruct"] = get_model_MPT7B8k
models["mosaicml/mpt-30b-instruct"] = get_model_MPT30B

# Tokenize (if specificities in the way models call the tokenizer)

# Inference (if specificities in the way models call the generate function)

# Treating the output (to remove the input if present in the output as well as the end of text token for example)

def treat_output_MPT(output): # Differs if it is N-shot. Here, it is 0-shot
    occ_1 = output.find("### Assistant: ")
    output = output[occ_1+15:]
    if output.find('<|endoftext|>')!=-1:
        output = output[:-14]
    return output

treat_output["mosaicml/mpt-7b-instruct"] = treat_output_MPT
treat_output["Trelis/mpt-7b-instruct-hosted-inference-8bit"] = treat_output_MPT
treat_output["mosaicml/mpt-7b-8k-instruct"] = treat_output_MPT
treat_output["mosaicml/mpt-30b-instruct"] = treat_output_MPT

#### Falcon

#### Bloom

#### Llama